In [4]:
import os
import pandas as pd
import cv2
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

data_dir = 'GTSRB\Training'
#GTSRB-training_fixed\GTSRB\Training

In [5]:
def preprocess(img):
    return cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)

In [6]:
list_images = []
output = []

for dir in os.listdir(data_dir):
    i = 0
    if dir == '.DS_Store' :
        continue
    inner_dir = os.path.join(data_dir, dir)
    csv_file = pd.read_csv(os.path.join(inner_dir,"GT-" + dir + '.csv'), sep=';')
    for row in csv_file.iterrows():
        if i < 100:
            img_path = os.path.join(inner_dir, row[1].Filename)
            img = cv2.imread(img_path)
            img = img[row[1]['Roi.X1']:row[1]['Roi.X2'],row[1]['Roi.Y1']:row[1]['Roi.Y2'],:]
            img = preprocess(img)
            list_images.append(img)
            output.append(row[1].ClassId)
        i += 1
        
print(np.shape(list_images))

(4300, 64, 64, 3)


In [78]:
# img_rgb = cv2.cvtColor(list_images[0], cv2.COLOR_BGR2RGB)
# plt.imshow(img_rgb)
# plt.show()
# img_rgb = cv2.cvtColor(list_images[4101], cv2.COLOR_BGR2RGB)
# plt.imshow(img_rgb)
# print(output[4101])

In [79]:
input_array = np.stack(list_images)
train_y = keras.utils.to_categorical(output)

randomize = np.arange(len(input_array))
np.random.shuffle(randomize)

x = input_array[randomize]
y = train_y[randomize]

In [80]:
split_size = int(x.shape[0]*0.6)
train_x, val_x = x[:split_size], x[split_size:]
train1_y, val_y = y[:split_size], y[split_size:]

split_size = int(val_x.shape[0]*0.5)
val_x, test_x = val_x[:split_size], val_x[split_size:]
val_y, test_y = val_y[:split_size], val_y[split_size:]

In [89]:
epochs = 10
batch_size = 16

input_shape = Input(shape=(32, 32,3))

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(64,64,3), padding='same'),
    BatchNormalization(),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.2),
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.2),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.2),
    
    Flatten(),
    Dense(units=2048, activation='relu'),
#     Dropout(0.2),
    Dense(units=1024, activation='relu'),
#     Dropout(0.2),
    Dense(units=128, activation='relu'),
#     Dropout(0.2),
    Dense(units=43, input_dim=2048, activation='softmax'),
    
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 1e-4), metrics=['accuracy'])
trained_model_conv = model.fit(train_x.reshape(-1,64,64,3), train1_y, epochs=epochs, batch_size=batch_size, validation_data=(val_x, val_y))

Epoch 1/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 46s 243ms/step - accuracy: 0.3067 - loss: 2.8111 - val_accuracy: 0.5360 - val_loss: 2.2111
Epoch 2/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 247ms/step - accuracy: 0.8969 - loss: 0.4140 - val_accuracy: 0.8907 - val_loss: 0.5304
Epoch 3/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 249ms/step - accuracy: 0.9680 - loss: 0.1202 - val_accuracy: 0.9349 - val_loss: 0.2951
Epoch 4/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 41s 251ms/step - accuracy: 0.9748 - loss: 0.0860 - val_accuracy: 0.9360 - val_loss: 0.2996
Epoch 5/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 245ms/step - accuracy: 0.9942 - loss: 0.0352 - val_accuracy: 0.9186 - val_loss: 0.3187
Epoch 6/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 39s 241ms/step - accuracy: 0.9894 - loss: 0.0429 - val_accuracy: 0.9384 - val_loss: 0.2808
Epoch 7/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 246ms/step - accuracy: 0.9789 - loss: 0.0731 - val_accuracy: 0.9465 - val_loss: 0.2114
Epoch 8/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 244ms/step - accuracy: 0.9919 - loss: 0

In [90]:
model.evaluate(test_x, test_y)

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.9814 - loss: 0.0740


[0.07296526432037354, 0.9790697693824768]